In [ ]:
import sys, os
sys.path.insert(0, '/home/niell_lab/Documents/GitHub/FreelyMovingEphys/')
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm
import matplotlib.gridspec as gridspec
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from scipy.stats import linregress
from matplotlib.backends.backend_pdf import PdfPages
import cv2
import xarray as xr
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
%matplotlib inline
mpl.rcParams.update({'font.size':10})

In [ ]:
%matplotlib inline

In [ ]:
train_psth = np.load('/home/niell_lab/Data/freely_moving_ephys/eye_movements/cross validation/June22/train_psth1.npy')
test_psth = np.load('/home/niell_lab/Data/freely_moving_ephys/eye_movements/cross validation/June22/test_psth1.npy')
good_inds = np.load('/home/niell_lab/Data/freely_moving_ephys/eye_movements/cross validation/June22/good_inds.npy')

In [ ]:
def get_direction_pref(left, right):
    evok_left = left[1000:1250]
    evok_right = right[1000:1250]
    
    ind = np.argmax([np.max(np.abs(evok_left)), np.max(np.abs(evok_right))])
    pref = ['left','right'][ind]
    nonpref = ('left' if pref=='right' else 'right')
    
    return pref, nonpref
    
def normalize_psth(psth, raw_pref=None, baseline_val=None):
    if raw_pref is None:
        raw_pref = psth.copy()
    if baseline_val is None:
        baseline_val = np.nanmean(psth[0:800].astype(float))
    norm_psth = (psth - baseline_val) / np.nanmax(raw_pref[750:1250].astype(float))
    return norm_psth

In [ ]:
norm_train = np.zeros([len(good_inds),2001])
norm_test = np.zeros([len(good_inds),2001])
for i, ind in enumerate(good_inds):
    norm_train[i,:] = normalize_psth(train_psth[ind,:])
    norm_test[i,:] = normalize_psth(test_psth[ind,:])

In [ ]:
psth_bins = np.arange(-1,1.001,1/1000)

In [ ]:
def calc_latency(psth):
    # use norm PSTH
    ind = np.argmax(psth[1025:1250])+1025 # was 1000:1170
    peakT = psth_bins[ind]
    val = psth[ind]
    return peakT, val

In [ ]:
train_peakT = np.zeros(np.size(norm_train,0))
test_peakT = np.zeros(np.size(norm_test,0))
for i in range(np.size(norm_train,0)):
    train_peakT[i], _ = calc_latency(norm_train[i,:])
    test_peakT[i], _ = calc_latency(norm_test[i,:])

In [ ]:
# sort peak times
order = np.argsort(train_peakT)

sort_train_psths = norm_train[order,:].copy()
sort_test_psths = norm_test[order,:].copy()

In [ ]:
def drop_nan_along(x, axis=1):
    # axis=1 will drop along columns (i.e. any rows with NaNs will be dropped)
    x = x[~np.isnan(x).any(axis=axis)]
    return x

In [ ]:
def plot_tempseq(panel, tseq, return_img=False, freev=None):
    tseq = drop_nan_along(tseq, 1)
    panel.set_xlabel('msec')
    panel.set_ylim([np.size(tseq,0),0])
    vmin = -0.75; vmax = 0.75
    if freev is not None:
        vmin = -freev
        vmax = freev
    img = panel.imshow(tseq, cmap='coolwarm', vmin=vmin, vmax=vmax)
    panel.set_xlim([800,1400])
    panel.set_xticks(np.linspace(800,1400,4), labels=np.linspace(-200,400,4).astype(int))
    panel.vlines(1000, 0, np.size(tseq,0), color='k', linestyle='dashed', linewidth=1)
    panel.set_aspect(2.8)
    if return_img:
        return img

In [ ]:
fig, [ax0,ax1] = plt.subplots(1,2,figsize=(4,4), dpi=300)

ax0_img = plot_tempseq(ax0, sort_train_psths)

ax1_img = plot_tempseq(ax1, sort_test_psths)
ax1.set_yticklabels([])

ax0.set_title('train')
ax1.set_title('test')

# fig.savefig('/home/niell_lab/Desktop/crossval.pdf')

In [ ]:
plt.figure(figsize=(2,2), dpi=300)
plt.plot(train_peakT[(train_peakT>.025) * (train_peakT<.250)],
         test_peakT[(test_peakT>.025) * (test_peakT<.250)], 'k.', markersize=3)
plt.xlabel('train latency (msec)'); plt.ylabel('test latency (msecs)')
plt.plot([0.02,.250], [0.02,.250], linestyle='dashed', color='tab:red', linewidth=1)
plt.xlim([.02, .20]); plt.ylim([.02, .250])
plt.xticks(np.linspace(0.020,.250,4), labels=np.linspace(20,250,4).astype(int))
plt.yticks(np.linspace(0.020,.250,4), labels=np.linspace(20,250,4).astype(int))

# plt.savefig('/home/niell_lab/Desktop/crossval_peakT.pdf', pad_inches=3)

In [ ]:
maxcc = np.zeros([len(sort_train_psths)])*np.nan
for i in range(len(sort_train_psths)):
    
    train = sort_train_psths[i,:].copy()
    test = sort_test_psths[i,:].copy()
    
    r = np.corrcoef(train[1000:1250], test[1000:1250])
    maxcc[i] = r[0,1]**2

In [ ]:
fig, ax0 = plt.subplots(1,1,figsize=(2.5,1.5), dpi=300)

weights = np.ones_like(maxcc) / float(len(maxcc))
n,_,_ = ax0.hist(maxcc, color='grey', bins=np.linspace(-1,1,21), weights=weights)
# ax0.set_xlabel('gaze shift cc');
ax0.set_ylabel('frac. cells')
ax0.set_xticks(np.arange(-1,1,3),labels=[])
ax0.plot([0,0], [0, .22], color='k', linewidth=1, linestyle='dashed')
ax0.set_ylim([0,.22])

fig.tight_layout()
fig.savefig('/home/niell_lab/Desktop/mouse_crossval_gazeshift_correlation.pdf')

In [ ]:
from scipy.stats import spearmanr

In [ ]:
spearmanr(train_peakT, test_peakT)

In [ ]:
res.rvalue

In [ ]:
0.8578792694126758**2

In [ ]:
np.linspace(0.035,.2,4)

In [ ]:
np.linspace(35,200,4).astype(int)